In [18]:
# https://www.semanticpartners.com/post/a-triple-store-rag-retriever
# https://api.python.langchain.com/en/latest/_modules/langchain/chains/graph_qa/sparql.html
# https://python.langchain.com/api_reference/community/chains/langchain_community.chains.graph_qa.sparql.GraphSparqlQAChain.html


In [19]:
from langchain_core.retrievers import BaseRetriever
from langchain.schema import Document
from typing import List
from SPARQLWrapper import SPARQLWrapper, JSON

class MillenniumDBRetriever(BaseRetriever):
    endpoint: SPARQLWrapper 
    
    def __init__(self, endpoint_url: str):
        super().__init__(endpoint=SPARQLWrapper(endpoint_url))
        self.endpoint.setReturnFormat(JSON)

    def _get_relevant_documents(self) -> List[Document]:

        sparql_query = f"""
                SELECT ?s ?p ?o
                WHERE {{
                    ?s ?p ?o .
                    FILTER(regex(str(?s), "A1327600018", "i") || regex(str(?o), "A1327600018", "i"))
                }}
        """

        self.endpoint.setQuery(sparql_query)
        results = self.endpoint.queryAndConvert()
        print(results)
        return results["results"]["bindings"]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        return self._get_relevant_documents(query)


In [20]:
from langchain.chains import GraphSparqlQAChain
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_community.graphs import RdfGraph

print("Inicializando RdfGraph...")

graph = RdfGraph(
    source_file='/Users/lucag/OneDrive/Escritorio/Facultad/LIDI/grafo_la_plata.ttl',
    standard="rdf",
)
print("RdfGraph creado correctamente.")

prompt = PromptTemplate.from_template(''' Write a SPARQL SELECT query for querying a graph database.
  The ontology schema delimited by triple backticks in Turtle format is:
  ```
  {schema}
  ```
  Use only the classes and properties provided in the schema to construct the SPARQL query.
  Do not use any classes or properties that are not explicitly provided in the SPARQL query.
  Include all necessary prefixes, always include these :
  
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX inmo: <http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#>
    PREFIX avis: <https://raw.githubusercontent.com/fdioguardi/pronto/main/ontology/pronto.owl#>
    PREFIX : <http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#>
    PREFIX time: <http://www.w3.org/2006/time#>
    PREFIX sioc: <http://rdfs.org/sioc/ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rec: <https://w3id.org/rec#>
    PREFIX gr: <http://purl.org/goodrelations/v1#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX pronto: <https://raw.githubusercontent.com/fdioguardi/pronto/main/ontology/pronto.owl#>

  Do not include any explanations or apologies in your responses.
  Do not use this symbol `.
  Do not include any text except the SPARQL query generated.
  The question delimited by triple backticks is:
  """
  {prompt}
  """
 ''')
print("PromptTemplate creado.")

# , sparql_select_prompt=prompt
print("Cargando esquema RDF...")
graph.load_schema()
print("Esquema RDF cargado.")

print("Inicializando GraphSparqlQAChain...")
chainSparQL = GraphSparqlQAChain.from_llm(
    ChatOllama(model="phi3:3.8b", temperature=0), graph=graph, allow_dangerous_requests=True, verbose=True, return_sparql_query=True
)
print("GraphSparqlQAChain inicializado.")

question= "¿Cómo es el nombre de la ciudad donde está ubicado el terreno con id A1327600018?"

print("Invocando la cadena para generar la consulta SPARQL...")
sparql_result = chainSparQL.invoke(question)
print("Consulta SPARQL generada:")
sparql_query= sparql_result["sparql_query"]
print(sparql_query)

Inicializando RdfGraph...
RdfGraph creado correctamente.
PromptTemplate creado.
Cargando esquema RDF...
Esquema RDF cargado.
Inicializando GraphSparqlQAChain...
GraphSparqlQAChain inicializado.
Invocando la cadena para generar la consulta SPARQL...


> Entering new GraphSparqlQAChain chain...
Identified intent:
SELECT
Generated SPARQL:
```sparql
SELECT ?city WHERE {
    GRAPH <http://example.org/montology> {
        GRAPH <http://raw.githubusercontent.com/fdioguardi/pronto/main/ontology/pronto.owl#size_type> {
            a wd:Q57 ; # Wikidata item for "Person" class, assuming the city is associated with an individual in this graph
            rdfs:label "A1327600018"^^xsd:integer ,
            ?city .
        }
    }
}
```


ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found '`'  (at char 0), (line:1, col:1)

In [ ]:
MILLENNIUMDB_ENDPOINT = "http://localhost:1234/sparql" 

graph_retriever = MillenniumDBRetriever(endpoint_url=MILLENNIUMDB_ENDPOINT)
triples_text = graph_retriever._get_relevant_documents()
print(triples_text)

{'head': {'vars': ['s', 'p', 'o']}, 'results': {'bindings': [{'s': {'type': 'uri', 'value': 'https://raw.githubusercontent.com/fdioguardi/pronto/main/ontology/pronto.owl#site2'}, 'p': {'type': 'uri', 'value': 'http://rdfs.org/sioc/ns#space_of'}, 'o': {'type': 'uri', 'value': 'http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#listing_site2_A1327600018'}}, {'s': {'type': 'uri', 'value': 'http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#agent_Goncebate%20Loscalzo%20Negocios%20Inmobiliarios'}, 'p': {'type': 'uri', 'value': 'http://xmlns.com/foaf/0.1/made'}, 'o': {'type': 'uri', 'value': 'http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#listing_site2_A1327600018'}}, {'s': {'type': 'uri', 'value': 'http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#account_site2_244713448'}, 'p': {'type': 'uri', 'value': 'http://rdfs.org/sioc/ns#creator_of'}, 'o': {'type': 'uri', 'value': 'http://www.semanticweb.org/luciana/ontologies/2024/8/inmont

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

prompt = ChatPromptTemplate.from_template(
"""
Using only the given Context below, made of RDF-like triples, use it to answer the Question. Answer briefly.

*** Context ***
{triples}

*** Question ***
{question}

""")

llm = ChatOllama(model="llama3", temperature=0, verbose=True)

chain = prompt | llm
question= "¿Cómo es el nombre de la ciudad donde está ubicado el terreno con id A1327600018?"

input_data = {
    "question": question,
    "triples": triples_text
}
answer = chain.invoke(input_data)
print(answer)


content='A partir del contexto proporcionado, podemos identificar las siguientes triples relevantes:\n\n* `http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#listing_site2_A1327600018` `http://purl.org/dc/elements/1.1/date` `"2023-01-30T00:00:00"`\n* `http://www.semanticweb.org/luciana/ontologies/2024/8/inmontology#listing_site2_A1327600018` `http://www.w3.org/2000/01/rdf-schema#label` `"Terreno - Mar Del Plata"`\n\nLa triple que describe el título del listing indica que se trata de un "Terreno" ubicado en "Mar Del Plata". Por lo tanto, la respuesta a la pregunta es: **Mar Del Plata**.' additional_kwargs={} response_metadata={'model': 'llama3', 'created_at': '2025-08-25T20:53:17.363522906Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3613511685, 'load_duration': 11921060, 'prompt_eval_count': 2048, 'prompt_eval_duration': 404000000, 'eval_count': 182, 'eval_duration': 3196000000} id='run-0e20569f-3b1c-4d95-b094-